In [2]:
# ========================================================
# 1. 라이브러리 및 설정
# ========================================================
!pip install transformers seqeval datasets pandas

import torch
import pandas as pd
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification
from seqeval.metrics import classification_report
from sklearn.model_selection import train_test_split
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')
MODEL_PATH = '/content/drive/MyDrive/aero_bert_model'

# ========================================================
# 2. 데이터 로드 및 전처리 (Training과 동일한 로직)
# ========================================================
# 2-1. 항공사 DB 로드 (평가 데이터 라벨링용)
try:
    airline_df = pd.read_csv('data/airlines_list.csv', encoding='cp949')
    full_airline_list = airline_df['영문항공사명'].dropna().astype(str).tolist()
    refined_airlines = set()
    for name in full_airline_list:
        parts = name.split()
        if len(parts) > 0:
            refined_airlines.add(parts[0])
            refined_airlines.add(name)
    stop_words = {'Air', 'Airlines', 'Airways', 'Aviation', 'International', 'Limited', 'Co.', 'Inc.', 'The'}
    final_airline_list = [w for w in refined_airlines if w not in stop_words and len(w) > 2]
    print("✅ 항공사 DB 로드 완료")
except:
    print("⚠️ 항공사 DB 로드 실패 (기본 리스트 사용)")
    final_airline_list = ['Korean', 'Asiana', 'Jeju', 'Jin', 'Cathay', 'Delta']

# 2-2. 뉴스 데이터 로드
df = pd.read_csv('data/aviation_data.csv')
df = df[df['source'] == 'AeroRoutes']

# 2-3. 자동 라벨링 함수 (Training과 동일)
def auto_label_text(text):
    text = str(text)
    tokens = text.split()
    ner_tags = ['O'] * len(tokens)

    date_pattern = r'^\d{1,2}[A-Z]{3}\d{2}$'
    aircraft_pattern = r'^(A3\d{2}|B7\d{2}|7\d{7}|[A-Z]?\d{3}[a-z]?(-[A-Z0-9]+)?)$'

    for i, token in enumerate(tokens):
        clean_token = re.sub(r'[^a-zA-Z0-9]', '', token)
        if re.match(date_pattern, clean_token): ner_tags[i] = 'B-DATE'
        elif re.match(aircraft_pattern, clean_token): ner_tags[i] = 'B-AIRCRAFT'
        elif ('–' in token or '-' in token) and len(token) > 3 and not re.match(aircraft_pattern, clean_token): ner_tags[i] = 'I-ROUTE'
        elif clean_token in final_airline_list: ner_tags[i] = 'B-AIRLINE'
    return tokens, ner_tags

data = []
for content in df['content']:
    tokens, tags = auto_label_text(content)
    data.append({'tokens': tokens, 'ner_tags': tags})

# 2-4. 평가용 데이터셋 분할 (Validation Set만 사용)
_, val_data = train_test_split(data, test_size=0.15, random_state=42)
print(f"✅ 평가용 데이터(Validation Set) 개수: {len(val_data)}개")

# ========================================================
# 3. 모델 및 토크나이저 로드
# ========================================================
tokenizer = BertTokenizerFast.from_pretrained(MODEL_PATH)
model = BertForTokenClassification.from_pretrained(MODEL_PATH)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

# 라벨 맵핑 정보 로드
id2label = model.config.id2label
label2id = model.config.label2id
label_list = list(label2id.keys()) # ['B-AIRCRAFT', 'B-AIRLINE', ...]

# ========================================================
# 4. 평가 실행 (전체 Validation Set)
# ========================================================
print("\n=== 🚀 전체 평가 시작 (잠시만 기다려주세요) ===")

true_labels = []
pred_labels = []

# 배치 단위 말고 하나씩 정확하게 평가
with torch.no_grad():
    for item in val_data:
        tokens = item['tokens']
        tags = item['ner_tags'] # 정답 라벨

        # 토크나이징 (정답 라벨 위치 맞추기)
        tokenized_inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True, padding=True)
        word_ids = tokenized_inputs.word_ids()

        inputs = tokenized_inputs.to(device)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)

        # [핵심] Subword 토큰을 다시 단어 단위 라벨로 변환
        # 특수 토큰([CLS], [SEP])이나 Subword(##)는 제외하고 첫 토큰의 예측값만 가져옴
        curr_pred = []
        curr_true = []

        previous_word_idx = None
        for idx, word_idx in enumerate(word_ids):
            if word_idx is None: continue # 특수 토큰 무시
            if word_idx != previous_word_idx: # 새 단어의 시작
                # 예측값 저장
                pred_tag_id = predictions[0][idx].item()
                curr_pred.append(id2label[pred_tag_id])
                # 실제값 저장
                curr_true.append(tags[word_idx])
            previous_word_idx = word_idx

        true_labels.append(curr_true)
        pred_labels.append(curr_pred)

# ========================================================
# 5. 결과 출력 (Classification Report)
# ========================================================
print("\n" + "="*50)
print("📊 모델 성능 평가 리포트 (Validation Set)")
print("="*50)

# seqeval 라이브러리가 알아서 F1, Precision, Recall 계산해줍니다.
report = classification_report(true_labels, pred_labels)
print(report)

# ========================================================
# 6. 정성 평가 (실제 예측 샘플 확인)
# ========================================================
print("\n" + "="*50)
print("👀 실제 예측 샘플 (랜덤 3개)")
print("="*50)

for i in range(3):
    sample = val_data[i]
    text = " ".join(sample['tokens'])
    true = sample['ner_tags']
    pred = pred_labels[i]

    print(f"\n[Sample {i+1}]")
    print(f"문장: {text[:100]}...") # 너무 기니까 앞부분만

    # 틀린 부분만 찾아서 보여주기 (없으면 완벽)
    diffs = []
    for t, p, token in zip(true, pred, sample['tokens']):
        if t != p and t != 'O': # 정답이 있는데 틀린 경우
            diffs.append(f"{token}(정답:{t} -> 예측:{p})")

    if not diffs:
        print("✅ 완벽하게 예측함!")
    else:
        print(f"⚠️ 차이점: {', '.join(diffs)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 항공사 DB 로드 완료
✅ 평가용 데이터(Validation Set) 개수: 150개


The tokenizer you are loading from '/content/drive/MyDrive/aero_bert_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.



=== 🚀 전체 평가 시작 (잠시만 기다려주세요) ===

📊 모델 성능 평가 리포트 (Validation Set)
              precision    recall  f1-score   support

    AIRCRAFT       0.99      0.97      0.98       333
     AIRLINE       0.93      0.94      0.93       367
        DATE       0.93      0.96      0.95       293
       ROUTE       0.87      0.91      0.89       213

   micro avg       0.93      0.95      0.94      1206
   macro avg       0.93      0.95      0.94      1206
weighted avg       0.94      0.95      0.94      1206


👀 실제 예측 샘플 (랜덤 3개)

[Sample 1]
문장: Published at 0500PDT / 1200GMT 20OCT25Alaska Airlines in Northern summer 2026 season plans to expand...
✅ 완벽하게 예측함!

[Sample 2]
문장: Published at 1701PDT 03OCT25 / 0001GMT 04OCT25Mexican carrier Aerus tomorrow (04OCT25) launches new ...
✅ 완벽하게 예측함!

[Sample 3]
문장: Published at 1400PDT / 2100GMT 03OCT25Boliviana de Aviacion (BoA) in the last few days revised plann...
✅ 완벽하게 예측함!
